# Clasificación de Rangos de Edad con PCA + KNN y Webcam

In [13]:
import os
import cv2
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

from FaceDetectors import FaceDetector
from FaceNormalizationUtils import Normalization

In [ ]:
def age_to_range(age):
    age = int(age)
    if age <= 12:
        return "Niño"
    elif age <= 19:
        return "Adolescente"
    elif age <= 35:
        return "Joven"
    elif age <= 60:
        return "Adulto"
    else:
        return "Anciano"

def load_age_dataset_ranged(base_path, img_size=(100,100)):
    X, y = [], []
    folders = sorted([f for f in os.listdir(base_path) if f.isdigit()], key=lambda x: int(x))
    for folder in folders:
        age = int(folder)
        age_range = age_to_range(age)
        folder_path = os.path.join(base_path, folder)
        for fname in os.listdir(folder_path):
            path = os.path.join(folder_path, fname)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, img_size)
            X.append(img.flatten())
            y.append(age_range)
    return np.array(X), np.array(y)


In [ ]:
def compute_pca(X, num_components=100):
    mean = np.mean(X, axis=0)
    Xc = X - mean
    cov = np.cov(Xc.T)
    eigvals, eigvecs = np.linalg.eigh(cov)
    idx = np.argsort(eigvals)[::-1]
    eigvecs = eigvecs[:, idx[:num_components]]
    return eigvecs, mean


In [4]:

def run_kfold(X_pca, y, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    accs = []
    for tr, te in kf.split(X_pca):
        Xtr, Xte = X_pca[tr], X_pca[te]
        ytr, yte = y[tr], y[te]
        clf = KNeighborsClassifier(n_neighbors=5)
        clf.fit(Xtr, ytr)
        pred = clf.predict(Xte)
        accs.append(accuracy_score(yte, pred))
    return accs


In [ ]:
# Ruta del dataset (MODIFICA ESTA)
dataset_path = "Dataset"

X, y = load_age_dataset_ranged(dataset_path)
print("Dataset:", X.shape, y.shape)

W, mean = compute_pca(X, num_components=100)
X_pca = (X - mean) @ W

accs = run_kfold(X_pca, y)
print("Accuracy medio:", np.mean(accs))

clf = KNeighborsClassifier(n_neighbors=7)
clf.fit(X_pca, y)

print("Modelo final entrenado.")

Dataset: (9778, 10000) (9778,)
Accuracy medio: 0.5109432057699046
Modelo final entrenado.


In [6]:
def preprocess_webcam_frame(frame, img_size=(100,100)):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, img_size)
    return gray.flatten()

def predict_age_range(img_flat, W, mean, clf):
    img_pca = (img_flat - mean) @ W
    return clf.predict([img_pca])[0]


# Discriminante por edades

In [7]:
detector = FaceDetector()

cap = cv2.VideoCapture(0)
print("Presiona 'q' para salir.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir a RGB porque MTCNN usa imágenes RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detectar la cara más grande
    values = detector.SingleFaceEyesDetection(rgb, facedet='MTCNN', eyesdet='')
    
    if values is not None:
        (x, y, w, h), eyes, shape = values
        
        # Recortar la cara detectada
        face_crop = rgb[y:y+h, x:x+w]

        # Preprocesar la cara para PCA
        try:
            face_gray = cv2.cvtColor(face_crop, cv2.COLOR_RGB2GRAY)
            face_gray = cv2.resize(face_gray, (100,100))
            face_flat = face_gray.flatten()

            # Proyectar en PCA y predecir
            pred_range = predict_age_range(face_flat, W, mean, clf)

            if pred_range == "Niño":
                color = (255, 0, 0)
            elif pred_range == "Adolescente":
                color = (0, 255, 255)
            elif pred_range == "Joven":
                color = (0, 255, 0)
            elif pred_range == "Adulto":
                color = (0, 0, 255)
            else:
                color = (128, 0, 128)


            # Mostrar recuadro de detección
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

            # Mostrar texto
            cv2.putText(frame, f"Rango: {pred_range}", (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color , 2)
        except:
            pass

    cv2.imshow("Clasificador Edad", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Presiona 'q' para salir.


# Filtro por edades

In [8]:
# -------------------------------------------------------------
# Cargar PNGs con canal alfa
# -------------------------------------------------------------
def load_png(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(f"No se encontró {path}")
    return img

try:
    png_chupete = load_png("assets/chupete.png")
    png_gafas   = load_png("assets/gafas.png")
    png_gorra   = load_png("assets/gorra.png")
    png_baston  = load_png("assets/baston.png")
except FileNotFoundError as e:
    print(f"Error cargando recursos: {e}. Asegúrate de que la carpeta 'assets' exista.")
    exit()

In [9]:
# -------------------------------------------------------------
# Función de overlay con alfa 
# -------------------------------------------------------------
def overlay_alpha(frame, overlay, x, y):
    h, w = overlay.shape[:2]
    h_frame, w_frame = frame.shape[:2]

    # Coordenadas de recorte para el frame
    x1, y1 = max(0, x), max(0, y)
    x2, y2 = min(w_frame, x + w), min(h_frame, y + h)

    # Coordenadas de recorte para el overlay
    ox1 = x1 - x
    oy1 = y1 - y
    ox2 = w - (x + w - x2)
    oy2 = h - (y + h - y2)
    
    if x2 <= x1 or y2 <= y1:
        return frame

    # Extraer ROIs
    roi_frame = frame[y1:y2, x1:x2]
    roi_overlay = overlay[oy1:oy2, ox1:ox2]

    alpha = roi_overlay[:,:,3] / 255.0
    
    if roi_frame.shape[:2] != alpha.shape[:2]:
         return frame 

    # Alpha blending
    for c in range(3):
        roi_frame[:,:,c] = (roi_frame[:,:,c] * (1 - alpha) + 
                            roi_overlay[:,:,c] * alpha).astype(np.uint8)
        
    return frame

In [15]:
# -------------------------------------------------------------
# Detector y normalizador
# -------------------------------------------------------------
detector = FaceDetector()
norm = Normalization()

cap = cv2.VideoCapture(0)
print("\n--- 2. INICIANDO CÁMARA (Clasificador Edad + Accesorios) ---")
print("Presiona 'q' para salir.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    values = detector.SingleFaceEyesDetection(rgb, facedet='MTCNN', eyesdet='')

    if values is not None:
        (x, y, w, h), eyes, shape = values

        # ------------------------------
        # Extraer ojos de forma segura (lx, ly, rx, ry)
        # ------------------------------
        lx = ly = rx = ry = None
        if isinstance(eyes, dict):
            lx, ly = eyes["left"]
            rx, ry = eyes["right"]
        elif isinstance(eyes, (list, tuple)):
            if len(eyes) == 4:
                lx, ly, rx, ry = eyes
            elif len(eyes) >= 2 and isinstance(eyes[0], (list, tuple)):
                lx, ly = eyes[0]
                rx, ry = eyes[1]

        if None in (lx, ly, rx, ry):
            continue

        # ------------------------------
        # Normalizar para dist_eyes y angle
        # ------------------------------
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        try:
            norm.normalize_gray_img(gray, lx, ly, rx, ry, kind="FACE")
            dist_eyes = norm.distf_eyes
            angle = norm.anglef_eyes
        except AttributeError:
            # Fallback: calcular ángulo/distancia directamente
            dx = rx - lx
            dy = ry - ly
            angle = np.degrees(np.arctan2(dy, dx))
            dist_eyes = np.sqrt(dx*dx + dy*dy)

        # ------------------------------
        # Predicción de rango de edad
        # ------------------------------
        if W is not None and mean is not None and clf is not None:
            try:
                face_crop = cv2.cvtColor(rgb[y:y+h, x:x+w], cv2.COLOR_RGB2GRAY)
                crop_gray = cv2.resize(face_crop, (100,100))
                pred_range = predict_age_range(crop_gray.flatten(), W, mean, clf)
            except Exception as e:
                pred_range = "Adulto"
        else:
            pred_range = "Adulto"

        # ------------------------------
        # Seleccionar accesorio y posición (CORRECCIÓN DE POSICIÓN)
        # ------------------------------
        if pred_range == "Niño":
            png = png_chupete
            pos_norm = (35, 75)  # boca 
            size_norm = 20
        elif pred_range == "Adolescente":
            png = png_gafas
            pos_norm = (30, 43)  # ojos
            size_norm = 30
        elif pred_range == "Joven":
            png = png_gorra
            pos_norm = (20, 5)   # frente
            size_norm = 50
        else:
            png = png_baston
            pos_norm = (5, 60)   # lateral
            size_norm = 25

        # ------------------------------
        # Transformar el PNG (Coordenadas Normalizadas -> Originales)
        # -------------------------------------------------------------
        
        cx_o = (lx + rx) // 2
        cy_o = (ly + ry) // 2

        # Distancia estándar (asumiendo 26px entre ojos en el espacio 100x100)
        dist_std = 26 
        scale = dist_eyes / dist_std

        # Convertir posición normalizada a desplazamiento original (dx, dy)
        dx = (pos_norm[0] - 30) * scale # 30: centro x de referencia
        dy = (pos_norm[1] - 37) * scale # 37: centro y de referencia (entre ojos)
        cx = int(cx_o + dx)
        cy = int(cy_o + dy)

        # Redimensionar y Rotar PNG
        oh, ow = png.shape[:2]
        new_w = int(size_norm * scale * 2)
        scale_png = new_w / ow
        new_h = int(oh * scale_png)
        
        if new_w <= 0 or new_h <= 0:
            continue 

        png_scaled = cv2.resize(png, (new_w, new_h), interpolation=cv2.INTER_AREA)
        
        # Rotar 
        bgr = png_scaled[:,:,:3]
        alpha = png_scaled[:,:,3]
        M = cv2.getRotationMatrix2D((new_w//2, new_h//2), -angle, 1.0)
        bgr_rot = cv2.warpAffine(bgr, M, (new_w, new_h), borderMode=cv2.BORDER_TRANSPARENT)
        alpha_rot = cv2.warpAffine(alpha, M, (new_w, new_h), borderMode=cv2.BORDER_TRANSPARENT)
        png_rot = np.dstack([bgr_rot, alpha_rot])

        # Coordenadas superior izquierda (para overlay)
        x1 = cx - new_w // 2
        y1 = cy - new_h // 2

        # Superponer PNG
        frame = overlay_alpha(frame, png_rot, x1, y1)

    cv2.imshow("Edad + Accesorios", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


--- 2. INICIANDO CÁMARA (Clasificador Edad + Accesorios) ---
Presiona 'q' para salir.
